In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()


pkl_file = open('./boc/formatted/im_array')
im_array = pickle.load(pkl_file)
pkl_file.close()

In [5]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind
import sys

n_pcs = range(10, 110, 5)
non_lin = [2]
non_lin_dict = {1: "sigmoid", 2: "relu"}
alpha = 0

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 60)
        
        w, o, s  = model.get_params()
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test

0%                100%
[                    ]

testing non-linearity:  relu ... testing  10  pc's...


[#                   ] | ETA: 00:00:00

new best model for cell  517442753 . r2 test:  -0.00989252557541
new best model for cell  517444738 . r2 test:  -0.0193311471369
new best model for cell  517443331 . r2 test:  0.0473550471913
new best model for cell  517442436 . r2 test:  -0.0157361450307
new best model for cell  517445575 . r2 test:  0.033766990586
new best model for cell  517444426 . r2 test:  0.0388195650162
new best model for cell  517442471 . r2 test:  -0.0263765159891
new best model for cell  517443181 . r2 test:  0.0396075638719
new best model for cell  517443087 . r2 test:  0.0295028852814
new best model for cell  517446833 . r2 test:  0.0523760403828
new best model for cell  517443827 . r2 test:  0.0378457104075
new best model for cell  517443924 . r2 test:  0.0107795984228
new best model for cell  517443957 . r2 test:  0.00829872961617
new best model for cell  517442722 . r2 test:  0.0235566878996
new best model for cell  517446488 . r2 test:  0.0197038861131
new best model for cell  517442675 . r2 test:  0.0

[##                  ] | ETA: 00:25:04

...

[##################  ] | ETA: 00:17:46

testing non-linearity:  relu ... testing  100  pc's...


[################### ] | ETA: 00:09:21

new best model for cell  517444738 . r2 test:  0.0662159251667
new best model for cell  517445575 . r2 test:  0.0815287124764
testing non-linearity:  relu ... testing  105  pc's...


[####################] | ETA: 00:00:00
Total time elapsed: 03:17:02


In [ ]:
#let's try this explainable variance dealio

im_array = np.array(im_array)
rishi_alex = {k: [] for k in data.keys()}

av_r = np.zeros_like(data[c])

for c in data.keys():
    dff = np.array(data[c]) 
    for image in range(118):
        idx  = np.where(im_array == image)
        ave = np.mean(dff[idx[0]])        
        av_r[idx] = ave
        
    slope, intercept, r_value, p_value, std_err = linregress(dff, av_r)
    rishi_alex[c] = r_value**2    

In [ ]:
plt.hist(rishi_alex.values());

print max(rishi_alex.values())

In [ ]:
#first let's calculate some aggregate statistics - and plot how explained variance goes with # of PC's
for c in data.keys():
    
    ra_var = rishi_alex[c]
    
    plt.subplot(221)
    for (l,  n) in scores_dict[c].keys():
        plt.scatter(n, scores_dict[c][(l, n)][0] / ra_var)
     
    plt.title('cell ' + str(c) + ' test $r^2$')
    
    plt.subplot(222)   
    for (l,  n) in scores_dict[c].keys():
        plt.scatter(n, scores_dict[c][(l, n)][1] / ra_var)
     
    plt.title('cell ' + str(c) + ' train $r^2$')
    plt.show()

In [8]:
pkl_file = open('./boc/formatted/LN_Relu', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()